# Introduction

Dans ce projet, nous avons implémenté un modèle de deep learning pour pouvoir détecter les machines malicieuses sur un réseau informatique. Nous avons utilisé le 'botnet dataset' de l'Université du Nouveau-Brunswick (UNB). Un botnet est un réseau informatique infecté par des personnes externes qui utilise la puissance de plusieurs ordinateurs pour effectuer des actions malveillantes. Pour commencer, j'ai lu des papiers de recherche sur la detéction de botnet à l'aide de l'intelligence artificielle. Par la suite, j'ai du faire le nettoyage des données ainsi que la selection des features à garder ainsi que la génération de features. Pour finir, j'ai utilisé la librairies keras pour faire mon modèle. Les résultats de mon modèle n'était pas à la hauteur de mes attentes, il faudrait donc évaluer des modèles différents dans le futur. Dans ce rapport, le code essentiel pour faire marcher le modèle ci-trouve pour le dataset qui m'a était fourni. Le code que j'ai travaillé durant la recherche se retrouve surtout dans le fichier old.py .

# Librairies

Lors de ce projet, j'ai pu toucher à plusieurs librairies connus dans le monde de science de donnée. Tout au long du projet, j'ai travaillé avec la librairie pandas pour manipuler les dataframes et créer les features de mon modèle. J'ai également utiliser la librairie sklearn ou scikit pour effectuer des fonctions mathématiques pour normaliser mes données. Pour finir, j'ai utilise la librairie keras pour implémenter mon modèle. 

In [ ]:
import keras
import pandas as pd
from keras import regularizers
from sklearn.preprocessing import StandardScaler

# Data cleansing

Durant cette partie de mon projet, j'ai vraiment pris le temps de me familiariser avec les différentes technologies de data science comme pandas. Il a fallu que j'effectue plusieurs opérations pour m'assurer que mes données soient sous la meilleur forme pour que mon modèle atteigne les meilleurs résultats. En partant, le dataset de l'UNB était déjà sous une très bonne forme. Pour commencer, il a fallu que j'assigne à chaque netflow une valeur de 1 ou 0 à son label pour savoir si celui-ci est malicieux ou sécure. L'Université fournit une liste des addresses malicieuses puis si celle-ci est dans le netflow en tant qu'une source ou une destination, j'assigne une valeur de 1 à cette rangée pour la désigner malicieuse. Au netflow sécure, j'assigne une valeur de 0. Puisqu'il y avait 2 colonnes avec des valeurs qui possédait des strings représentant la valeur infini, j'ai ignoré les rangées possédant cette valeur. J'ai également changé le type de ces colonnes pour avoir de meilleurs précisions à mes chiffres. Pour finir, j'ai drop les features inutiles à l'entraînement de mon modèle ainsi que les rangées contenant des valeurs nulles. 

In [ ]:
def rename_dataframe_column(dataframe_rename_columns):
    for column in dataframe_rename_columns.columns:
        new_column_name = column.replace(" ", "_").replace('/', '_').lower()
        # new_column_name = column.replace("/", "_")
        dataframe_rename_columns.rename(index=str, columns={column: new_column_name}, inplace=True)


def clean_dataset(dataset_to_clean):
    malicious_ips = ['192.168.2.112', '198.164.30.2', '192.168.2.113', '192.168.2.112', '147.32.84.180',
                     '147.32.84.140', '10.0.2.15',
                     '172.16.253.130', '172.16.253.240', '192.168.3.35', '172.29.0.116', '192.168.248.165',
                     '131.202.243.84',
                     '192.168.2.110', '192.168.1.103', '192.168.2.109', '147.32.84.170', '147.32.84.130',
                     '192.168.106.141',
                     '172.16.253.131', '74.78.117.238', '192.168.3.25', '172.29.0.109', '10.37.130.4', '192.168.5.122',
                     '192.168.4.118',
                     '192.168.4.120', '192.168.2.105', '147.32.84.150', '147.32.84.160', '192.168.106.131',
                     '172.16.253.129',
                     '158.65.110.24', '192.168.3.65', '172.16.253.132']

    
    # iterate through dataframe and set value
    for row in dataset_to_clean.itertuples():
        if ((dataset_to_clean.at[row.Index, 'src_ip'] in malicious_ips) or (
                dataset_to_clean.at[row.Index, 'dst_ip'] in malicious_ips)):
            dataset_to_clean.at[row.Index, 'label'] = 1
        else:
            dataset_to_clean.at[row.Index, 'label'] = 0

    # Drop row with infinity string value
    dataset_to_clean.drop(dataset_to_clean.loc[dataset_to_clean['flow_byts_s'] == "Infinity"].index, inplace=True)
    dataset_to_clean.drop(dataset_to_clean.loc[dataset_to_clean['flow_pkts_s'] == "Infinity"].index, inplace=True)

    # Pour drop colonne completement
    to_drop = ['flow_id', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'timestamp']

    to_drop_feature_selection = ['flow_id', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'timestamp']

    # inplace = true, fait en sorte que c est drop directement dans l objet
    dataset_to_clean.drop(to_drop_feature_selection, axis=1, inplace=True)

    # dataset.dropna(subset = ['Src IP','Src Port','Dst IP','Dst Port'])
    dataset_to_clean.dropna(how="any", axis=0, inplace=True)

    # cast flow pkts and
    dataset_to_clean[["flow_byts_s", "flow_pkts_s"]] = \
        dataset_to_clean[["flow_byts_s", "flow_pkts_s"]].apply(pd.to_numeric)

# méthode utilisé avec le nouveau dataset
def clean_dataset2(dataset_to_clean):
    to_drop_feature_selection = ['StartTime', 'Proto', 'SrcAddr', 'Sport', 'DstAddr', 'Dport','unix_time']
    dataset_to_clean.drop(to_drop_feature_selection, axis=1, inplace=True)

# Feature selection

Durant cette partie du projet, j'ai voulu essayer d'enlever quelques features qui pourraient être inutile. J'ai donc fait une matrice de corrélation entre les features pour pouvoir éliminer des features qui sont fortement corrélés entre eux. Après ce processus, j'ai conclu que cétait mieux de garder tous les features. Pour pouvoir le code pour la matrice de corrélation, référez-vous au notebook Feature Correlations.

# Feature Generation

Durant cette phase du projet, on m'a assigné la tâche de générer des features pour un autre projet. Je n'ai donc pas eu la chance de faire les features pour ce projet. Par contre, on m'a donné le fichier csv à utiliser qui contient des nouveaux features. Voici la liste de features qui a était ajouté.

### Connection-based features : 

(Using a rolling window for the previous n netflows when a given source or destination address appears in the traffic)

- For any of the flow records that SRCADDRESS has appeared within the last n flow records, average the bytes (A_TotBytes_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, average the packets (A_TotPkts_S)
- Number of apperance of SRCADDRESS in the last n/10 netflows (Nbr_App_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, count the distinct source ports (Dct_Sport_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, count the distinct destination ports (Dct_Dport_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, count the distinct source ips (Dct_SrcAddr_S)

- For any of the flow records that DSTADDRESS has appeared within the last n flow records, average the bytes (A_TotBytes_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, average the packets (A_TotPkts_D)
- Number of apperance of DSTADDRESS in the last n/10 netflows (Nbr_App_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, count the distinct source ports (Dct_Sport_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, count the distinct destination ports (Dct_Dport_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, count the distinct destination ips (Dct_DstAddr_D)

###  Time-based features : 

(Using a rolling window for the n previous minutes when a given source or destination address appears in the traffic). 

- For any of the flow records that SRCADDRESS has appeared within the last n minutes, average the bytes (A_TotBytes_S)
- For any of the flow records that SRCADDRESS has appeared within the last n minutes, average the packets (A_TotPkts_S)
- For any of the flow records that SRCADDRESS has appeared within the last n minutes, count the distinct source ports (Dct_Sport_S)
- For any of the flow records that SRCADDRESS has appeared within the last n minutes, count the distinct source ips (Dct_SrcAddr_S)
- Number of apperance of SRCADDRESS  within the last n/10 minutes (Nbr_App_S)

- For any of the flow records that DSTADDRESS has appeared within the last n minutes, average the bytes (A_TotBytes_D)
- For any of the flow records that DSTADDRESS has appeared within the last n minutes, average the packets (A_TotPkts_D)
- For any of the flow records that DSTADDRESS has appeared within the last n minutes, count the distinct source ports (Dct_Sport_D)
- For any of the flow records that DSTADDRESS has appeared within the last n minutes, count the distinct destination ips (Dct_DstAddr_D)
- Number of apperance of DSTADDRESS  within the last n/10 minutes (Nbr_App_D)


# Modèle

Mon modèle a était construit à l'aide de la librairie Keras. Celui-ci contient une layer de input de 18 neuronnes, une layer caché de 9 neuronnes ainsi qu'une layer de sortie de 1 neuronne. La fonction d'activation de la première couche est relu ainsi que celle de sortie est sigmoid puisque c'est un modèle binaire (retourne une valeur de 0 ou 1).

In [ ]:
# dataset_training = pd.read_csv('./ISCX_Botnet-Training.pcap_Flow_ubuntu.csv', encoding='utf-8', low_memory=False)
# dataset_testing = pd.read_csv('./ISCX_Botnet-Testing.pcap_Flow.csv', encoding='utf-8', low_memory=False)

dataset_training = pd.read_csv('unb-Training.csv', encoding='utf-8', low_memory=False)
dataset_testing = pd.read_csv('unb-testing.csv', encoding='utf-8', low_memory=False)

clean_dataset2(dataset_testing)
clean_dataset2(dataset_training)

# Split data
x_data_training = dataset_training.drop('Label', axis=1)
labels_training = dataset_training['Label']
x_data_testing = dataset_testing.drop('Label', axis=1)
labels_testing = dataset_testing['Label']

# Normalize data
std = StandardScaler()
x_data_training = std.fit_transform(x_data_training)
x_data_testing = std.fit_transform(x_data_testing)

# Create Model
model = keras.Sequential()
# Input Layer Layer and first hidden layer
model.add(keras.layers.Dense(9, activation='relu', input_shape=(18,), kernel_regularizer= regularizers.l2(0.01)))
model.add(keras.layers.Dropout(0.4))
# Output Layer
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001), metrics=["accuracy"])

es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Résultat

In [ ]:
model.fit(x_data_training, labels_training, epochs=2000, batch_size=500, shuffle= True,
                             validation_data=(x_data_testing, labels_testing), verbose=2)

predictions = model.predict(x_data_testing, batch_size=100, verbose=0)

print(predictions)

Comme on peut le voir, les résultats sont assez décevant. En training, le modèle est capable d'atteindre une accuracy de 99%, mais en testing celui-ci reste entre 40% et 50%. Cela peut être dû à plusieurs facteurs. J'ai essayé beaucoup d'optimisateurs différent avec des chiffres différents pours les champs suivants: learning rate, batch size en training et testing, ainsi que le nombre d'epochs. Dans le champs de val_loss, on voit qu'en testing le modèle est assez overfit même s'il se stabilise vers un certains chiffre. J'ai même ajouté un regularizer puis un dropout pour pouvoir essayer de régler ce problème. 

# Dans le futur

Dans le futur, plusieurs modifications ou ajout pourrait être fait à l'ensemble du projet. Pour commencer, il serait peut-être bon de garder les features fournit par l'UNB en plus de ceux que nous avons générer. Par la suite, il faudrait utiliser une autre librairie de modèle comme Tensorflow pour construire notre modèle. Une autre solution intéressante serait de faire un autre type de modèle, par exemple un modèle récurrent.